### Item 15: Know How Closures Interact with Variable Scope

In [ ]:
def sort_priority(values, group): 
    def helper(x):
        if x in group:
            print (0, x)
            return (0, x)  # 0 is priority group for sorting
        print(1, x)
        return (1, x)  # # 1 is normal group
    values.sort(key=helper)

In [ ]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

In [ ]:
sort_priority(numbers, group)

In [ ]:
print(numbers)

In [ ]:
print(group)

1. Python supports closures: 
    * functions that refer to variables from the scope in which they were defined. 
    * This is why the helper function is able to access the group argument to sort_priority.

2. Functions are first-class objects in Python, meaning 
    * you can refer to them directly, 
    * assign them to variables, 
    * pass them as arguments to other functions, 
    * compare them in expressions and if statements, etc. 
    * This is how the sort method can accept a closure function as the key argument.

3. Python has specific rules for comparing tuples. 
    * It first compares items in index zero, then index one, then index two, and so on. 
    * This is why the return value from the helper closure causes the sort order to have two distinct groups.
* There are three reasons why this function operates as expected:

* It'd be nice if the function returned whether higher priority items were seen at all.
* There is already a closure function for deciding which group each number is in.
* Use the closure to flip a flag when high-priority items are seen.
    * The function can return the flag value after it's been modified by the closure.

In [ ]:
def sort_priority2(numbers, group):
    found = False  # scope: outer function (containing function)
    def helper(x):
        if x in group:
            found = True  # scope: inner function (closure function)
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [ ]:
found = sort_priority2(numbers, group)

In [ ]:
found

In [ ]:
print(numbers)

* Problem 1 - `scoping bug`

    * The sorted results are correct, but the found result is wrong.
    * How could this happen?

* When you reference a variable in an expression, 
the Python interpreter will traverse the scope to resolve the reference in this order:
    1. The `current function’s scope`
    2. Any `enclosing scopes` (like other `containing functions`)
    3. The scope of the module that contains the code (also called the `global scope`) 
    4. The `built-in scope` (that contains functions like `len` and `str`)
    
* If none of these places have a defined variable with the referenced name, then a NameError exception is raised.

* Assigning a value to a variable works differently.
* The `assignment behavior` explains the wrong return value of `sort_priority` fuction.
    * The found variable is assigned to `True` in the `helper closure`.
    * The `closure's assignment` is treated as a new variable definition within helper, not as an assignment within `wort_priority2`.


#### Getting Data Out
* In Python 3, there is a special syntax for getting data out of a closure.
* The `nonlocal` statement makes it clear when data is being assigned out of a closure into another scope.
* Limit: `nonlocal` won't traverse up to the module-level scope (to avoid polluting globals).
* Do not use `nonlocal` for anything beyond simple functions.
    * The side effects of `nonlocal` can be hard to follow.

In [ ]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return(1, x)
    numbers.sort(key=helper)
    return found

In [ ]:
sort_priority3(numbers, group)

In [ ]:
print(numbers)

#### Write a Class!
* When the usage of `nonlocal` starts getting complicated, it's better to wrap the state in a helper class.
* Define a class that achieves the same result as the `nonlocal` approach.
    * It's a little longer, but is much easier to read.

In [ ]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)    

In [ ]:
sorter = Sorter(group)

In [ ]:
numbers.sort(key=sorter)
numbers

In [ ]:
# sorter has two attributes (group & found)
dir(sorter)

In [ ]:
for x in sorter.group:
    print(sorter.__call__(x))

In [ ]:
sorter.group

In [ ]:
sorter.found

In [ ]:
assert sorter.found is True

In [ ]:
numbers

### Things to Remember

* Closure functions can refer to variables from any of the scopes in which they were defined.
* By default, closures can’t affect enclosing scopes by assigning variables.
* In Python 3, use the nonlocal statement to indicate when a closure can modify a variable in its enclosing scopes.
* Avoid using nonlocal statements for anything beyond simple functions.